In [ ]:
import copy
import numpy as np
import pandas as pd

from experiments_2022 import DATASETS_PATH
from experiments_2022.zone_level_analysis import cleaning, base
from experiments_2022.zone_level_analysis.regression_functions import (
    general_regression_fn,
)
from experiments_2022.datasets import load_zones

In [ ]:
pd.set_option("display.max_rows", 10000)

In [ ]:
projects = copy.deepcopy(base.VAV_BUILDINGS)
projects.append("LAB-2")

In [ ]:
results = pd.DataFrame(
    index=list(range(10000)),
    columns=["Building", "VAV", "Minimum Airflow (cfm)", "Maximum Airflow (cfm)"],
)

i = 0
for project in projects:
    tloads_all = cleaning.clean_df(
        load_zones("2023", project, "zone-tloads"),
        "zone-tloads",
        only_business_hours=True,
        no_weekends=True,
    )
    airflowsp_all = cleaning.clean_df(
        load_zones("2023", project, "zone-airflowsp"),
        "zone-airflowsp",
        only_business_hours=True,
        no_weekends=True,
    )

    # common elems
    cols = tloads_all.columns.intersection(airflowsp_all.columns)
    idx = tloads_all.index.intersection(airflowsp_all.index)
    tloads_all = tloads_all.loc[idx, cols]
    airflowsp_all = airflowsp_all.loc[idx, cols]

    # for min, first try find times when tload = 0
    min = pd.Series(np.nan, index=cols)
    for vav in min.index:
        tload_vav = tloads_all[vav]
        idx = tload_vav[tload_vav == 0].index
        if len(idx) > 0:
            this_min = airflowsp_all.loc[idx, vav].median()
            if this_min > 0:
                min[vav] = this_min
    # for max, first try to find times when tload = 100
    max = pd.Series(np.nan, index=cols)
    for vav in max.index:
        tload_vav = tloads_all[vav]
        idx = tload_vav[tload_vav == 100].index
        if len(idx) > 0:
            this_max = airflowsp_all.loc[idx, vav].median()
            if this_max > 0 and this_max > min[vav]:
                max[vav] = this_max
    # next, try to use regression to find min and max
    mask = tloads_all > 0
    tloads_reg = tloads_all[mask]
    airflowsp_reg = airflowsp_all[mask]
    all_results = general_regression_fn(tloads_reg, airflowsp_reg)
    min_reg = -all_results["Slope Intercept"] / all_results["Slope X"]
    max_reg = (100 + all_results["Slope X"] * min) / all_results["Slope X"]
    for vav in min.index:
        if (np.isnan(min[vav])) and (min_reg[vav] > 0):
            min[vav] = min_reg[vav]
        if np.isnan(max[vav]) and (max_reg[vav] > 0) and (max_reg[vav] > min_reg[vav]):
            max[vav] = max_reg[vav]
    results.loc[i : i + len(min) - 1, "Building"] = project
    results.loc[i : i + len(min) - 1, "VAV"] = list(min.index)
    results.loc[i : i + len(min) - 1, "Minimum Airflow (cfm)"] = list(min.values)
    results.loc[i : i + len(min) - 1, "Maximum Airflow (cfm)"] = list(max.values)
    i = i + len(min) - 1
results = results.dropna(how="all")

In [ ]:
# results

In [ ]:
# len(results["Minimum Airflow (cfm)"][results["Minimum Airflow (cfm)"].isna()]) / len(results["Minimum Airflow (cfm)"])

In [ ]:
# len(results["Maximum Airflow (cfm)"][results["Maximum Airflow (cfm)"].isna()]) / len(results["Maximum Airflow (cfm)"])

In [ ]:
results.to_csv(DATASETS_PATH / "csvs" / "min_airflow_2023.csv")